In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../pycore/')
from simulate import RBA_result
from utils import extract_details_from_rxnid

In [2]:
df_cases = pd.read_csv('./input/data_list.txt', sep='\t')
cases = df_cases.data_id.to_list()
df_cases.index = df_cases.data_id.to_list()

In [3]:
df_enzmw = pd.read_csv('../../input/enz_mw_g_per_mmol_norxnmapped.txt', sep='\t')
df_enzmw.index = df_enzmw.enz_id.to_list()

In [4]:
df_compile = pd.DataFrame(columns=cases)

for case in cases:
    path_data = './datasets/' + case + '/'
    biom_id = df_cases.biomass_dilution_id[case]
    mu = df_cases.growth_rate[case]
    
    # Get measurable enzyme index
    path =  path_data + 'enz_from_proteome/enz_flux_calculation.txt' 
    res_esyn = RBA_result(biom_id=biom_id, twocol_format=True, warning=False);
    res_esyn.load_raw_flux(path)
    x = [k for k in list(res_esyn.raw_flux.keys()) if k.split('-')[0] == 'ENZLOAD']
    eloads = [k for k in x if res_esyn.raw_flux[k] > 0]
    
    rxns_eload = []
    for e in eloads:
        _,rxn,rdir,enz = extract_details_from_rxnid(e)
        rxns_eload.append(rxn)
    rxns_eload = sorted(list(set(rxns_eload)))
    
    # Get calculated flux
    path =  path_data + 'min_flux_sum/min_flux_sum.flux.txt' 
    res_metab = RBA_result(biom_id=biom_id)
    res_metab.load_raw_flux(path)
    res_metab.calculate_metabolic_flux()
    rxns_flux = list(res_metab.metabolic_flux.keys())
    
    # Get calculated flux corresponding to measurable enzyme
    rxns_idx = sorted(list(set(rxns_flux) & set(rxns_eload)))
    enzs_idx = []
    for e in eloads:
        _,rxn,rdir,enz = extract_details_from_rxnid(e)
        if rxn in rxns_flux:
            enzs_idx.append(enz)
    
    for enz in enzs_idx:
        df_compile.loc[enz,case] = res_esyn.raw_flux['ENZSYN-'+enz] / mu * df_enzmw.loc[enz,'MW (g/mmol)']

In [5]:
df_compile['id'] = df_compile.index.to_list()

In [6]:
df_rxnenz = pd.read_excel('./input/rxn_enz_pathway.xlsx')
for i in df_rxnenz.index:
    enzs = df_rxnenz.enzyme[i].split(',')
    for e in enzs:
        if e in df_compile.index:
            df_compile.loc[e, 'subsystem_group'] = df_rxnenz.loc[i, 'subsystem_group']
            df_compile.loc[e, 'subsystem'] = df_rxnenz.loc[i, 'subsystem']

In [7]:
cols = df_compile.columns.to_list()
df_compile = df_compile.loc[:, cols[-3:] + cols[:-3]]